# Data Processing

## Intent classification 테스크를 위한 데이터 처리

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/[강의자료] 한 번에 끝내는 딥러닝-인공지능 초격차 패키지 (수정, 배포 금지)_/Part 6. 딥러닝 실전 프로젝트_/Ch. 자연어 처리 실습/실습_코드/NLP_Chatbot
!pwd
!ls

/content/drive/MyDrive/[강의자료] 한 번에 끝내는 딥러닝-인공지능 초격차 패키지 (수정, 배포 금지)_/Part 6. 딥러닝 실전 프로젝트_/Ch. 자연어 처리 실습/실습_코드/NLP_Chatbot
/content/drive/MyDrive/[강의자료] 한 번에 끝내는 딥러닝-인공지능 초격차 패키지 (수정, 배포 금지)_/Part 6. 딥러닝 실전 프로젝트_/Ch. 자연어 처리 실습/실습_코드/NLP_Chatbot
/content/drive/MyDrive/[강의자료] 한 번에 끝내는 딥러닝-인공지능 초격차 패키지 (수정, 배포 금지)_/Part 6. 딥러닝 실전 프로젝트_/Ch. 자연어 처리 실습/실습_코드/NLP_Chatbot
1_1_embedding.ipynb	 2_1_NLG.ipynb				chatbot.model
1_2_intent_clsf.ipynb	 2_2_DialogManager.ipynb		chatbot.vocab
1_3_entity_recog.ipynb	 3_1_chit_chat_based_transformer.ipynb	data
1_4_ood_detection.ipynb  3_2_e2e_dialog.ipynb			src
1_5_NLU_BASE.ipynb	 4_1_dialog_system.ipynb


In [3]:
!pip install torch
!pip install --upgrade gensim==3.4.0
!pip install sentencepiece
!pip install pytorch-crf

     |████████████████████████████████| 22.2 MB 1.4 MB/s 
  Created wheel for gensim: filename=gensim-3.4.0-cp37-cp37m-linux_x86_64.whl size=23316680 sha256=f08b6e8476832af9573b4c01f5f3d1c66e5e5be9920000ebe9c27752a21d921b
  Stored in directory: /root/.cache/pip/wheels/de/a4/46/4e18f7d25915b16e0e790a5362e455aba6cadc486994806c05
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 1.2 MB 18.0 MB/s 


In [9]:
import os
import sys
import json
import torch
import random

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from tqdm import trange

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

#pip install sentencepiece
#pip install pytorch-crf
from src.dataset import Preprocessing
from src.model import EpochLogger, MakeEmbed

class MakeDataset:
    def __init__(self):
        
        self.intent_label_dir = "./data/dataset/intent_label.json"
        self.intent_data_dir = "./data/dataset/intent_data.csv"
        
        self.intent_label = self.load_intent_label()
        self.prep = Preprocessing()
    
    def load_intent_label(self):
        ''' 미리 만들어 둔 예측해야할 intent label 로드'''
        f = open(self.intent_label_dir, encoding="UTF-8") 
        intent_label = json.loads(f.read())
        self.intents = list(intent_label.keys())
        return intent_label
    
    def tokenize(self, sentence):
        ''' 띄어쓰기 단위로 tokenize 적용'''
        return sentence.split()
    
    def tokenize_dataset(self, dataset):
        ''' Dataset에 tokenize 적용'''
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset

    def make_intent_dataset(self, embed):
        ''' intent 분류를 위한 Dataset 생성'''
        intent_dataset = pd.read_csv(self.intent_data_dir) # 데이터 로딩

        labels = [self.intent_label[label] for label in intent_dataset["label"].to_list()] # label 
            
        intent_querys = self.tokenize_dataset(intent_dataset["question"].tolist()) # 사용자 발화 tokenize
        
        dataset = list(zip(intent_querys, labels)) # (사용자 발화, intent) 형태로 가공
        intent_train_dataset, intent_test_dataset = self.word2idx_dataset(dataset, embed) # word2index
        return intent_train_dataset, intent_test_dataset
    
    def word2idx_dataset(self, dataset ,embed, train_ratio = 0.8):
        embed_dataset = []
        question_list, label_list = [], []
        flag = True
        random.shuffle(dataset) #  훈련용과 검증용으로 나눌때 intent 편형이 나타나지 않도록 데이터 셔플
        for query, label in dataset :
            q_vec = embed.query2idx(query) # 사용자 발화 index화
            q_vec = self.prep.pad_idx_sequencing(q_vec) # 사용자 발화 최대길이까지 padding

            question_list.append(torch.tensor([q_vec]))
            label_list.append(torch.tensor([label]))

        x = torch.cat(question_list)
        y = torch.cat(label_list)

        # 학습용과 검증용으로 나누기
        x_len = x.size()[0]
        y_len = y.size()[0]
        if(x_len == y_len):
            train_size = int(x_len*train_ratio)
            
            train_x = x[:train_size]
            train_y = y[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]
            
            # TensorDataset으로 감싸기
            '''
             PyTorch의 TensorDataset은 tensor를 감싸는 Dataset입니다.

             인덱싱 방식과 길이를 정의함으로써 이것은 tensor의 첫 번째 차원을 따라 반복, 인덱스, 슬라이스를 위한 방법을 제공합니다.

             훈련할 때 동일한 라인에서 독립 변수와 종속 변수에 쉽게 접근할 수 있습니다.
            '''
            train_dataset = TensorDataset(train_x,train_y)
            test_dataset = TensorDataset(test_x,test_y)
            
            return train_dataset, test_dataset
            
        else:
            print("ERROR x!=y")
            

In [10]:
dataset = MakeDataset()

In [11]:
intent_dataset = pd.read_csv(dataset.intent_data_dir)

In [12]:
intent_dataset.head()

,question,label
0,야 먼지 알려주겠니,dust
1,아니 먼지 정보 알려주세요,dust
2,그 때 미세먼지 어떨까,dust
3,그 때 먼지 좋으려나,dust
4,미세먼지 어떨 것 같은데,dust


In [13]:
intent_dataset.groupby(['label']).count() 

,question
label,
dust,4997
restaurant,4997
travel,4999
weather,4999


In [14]:
embed = MakeEmbed()
embed.load_word2vec()

batch_size = 128

intent_train_dataset, intent_test_dataset = dataset.make_intent_dataset(embed)

# 한번의 iter당 Batch size의 x, y를 제공한다.
train_dataloader = DataLoader(intent_train_dataset, batch_size=batch_size, shuffle=True) 

test_dataloader = DataLoader(intent_test_dataset, batch_size=batch_size, shuffle=True)

In [15]:
intent_train_dataset.tensors

(tensor([[   8,  226,  219,  ...,    0,    0,    0],
         [  40,  635, 1240,  ...,    0,    0,    0],
         [ 449,  171,   16,  ...,    0,    0,    0],
         ...,
         [ 289,   15,  102,  ...,    0,    0,    0],
         [ 190,  219,   17,  ...,    0,    0,    0],
         [ 431,    2,    0,  ...,    0,    0,    0]]),
 tensor([1, 3, 1,  ..., 3, 1, 0]))

# Convolutional Neural Networks for Sentence Classification
## * Yoon Kim, New York University
### tensorflow code : https://github.com/SeonbeomKim/TensorFlow-TextCNN/blob/master/TextCNN.py

In [41]:
class textCNN(nn.Module):
    
    def __init__(self, w2v, dim, kernels, dropout, num_class):
        super(textCNN, self).__init__()
        # Word2vec으로 미리 학습해둔 임베딩 적용
        vocab_size = w2v.size()[0]
        emb_dim = w2v.size()[1]
        self.embed = nn.Embedding(vocab_size+2, emb_dim)
        self.embed.weight[2:].data.copy_(w2v)
        # self.embed.weight.requires_grad = False # 임베딩 레이어 학습 유무
        
        # 윈도우 사이즈가 다른 각각의 conv layer 를 nn.ModuleList로 저장
        # nn.Conv2d(in_channels, out_channels, kernel_size)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim, (w, emb_dim)) for w in kernels])
        #Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        #FC layer
        self.fc = nn.Linear(len(kernels)*dim, num_class)
        
    def forward(self, x):
        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [conv(emb_x) for conv in self.convs] # 각 사이즈 별 결과를 list로 저장, 
        print(con_x)
        #[(out_channels, conv결과 길이),...]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x] # 각 사이즈별 max_pool 결과 저장
        #[(256,1),...]

        fc_x = torch.cat(pool_x, dim=1) # concat하여 fc layer의 입력 형태로 만듬
        #(768,1)

        fc_x = fc_x.squeeze(-1) # 차원 맞추기
        #(768)
        fc_x = self.dropout(fc_x)
        logit = self.fc(fc_x)
        return logit

# 모델의 가중치 저장을 위한 코드
def save(model, save_dir, save_prefix, epoch):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, save_prefix)
    save_path = '{}_steps_{}.pt'.format(save_prefix, epoch)
    torch.save(model.state_dict(), save_path)

In [42]:
weights = embed.word2vec.wv.vectors # word2vec weight
weights = torch.FloatTensor(weights)

num_class = len(dataset.intent_label) 
model = textCNN(weights, 256, [3,4,5], 0.5, num_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [45]:
model

AttributeError: ignored

# Training

In [38]:
%%time
epoch = 10
prev_acc = 0
save_dir = "./data/pretraining/1_intent_clsf_model/"
save_prefix = "intent_clsf"
for i in range(epoch):
    steps = 0
    model.train() 
    #for data in train_dataloader:
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]
            logit = model.forward(x)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(logit, target) # loass function
            loss.backward()
            optimizer.step()

            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            tepoch.set_postfix(loss=loss.item(), accuracy= accuracy.numpy())
            
    model.eval() # weight 업데이트 금지
    steps = 0
    accuarcy_list = []
    #for data in test_dataloader:
    with tqdm(test_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]

            logit = model.forward(x)
            loss = F.cross_entropy(logit, target)
            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            accuarcy_list.append(accuracy.tolist())
            
            tepoch.set_postfix(loss=loss.item(), accuracy= sum(accuarcy_list)/len(accuarcy_list))
    
    # epoch 당 검증 셋의 정확도를 계산하고 이전 정확도 보다 높으면 저장     
    acc = sum(accuarcy_list)/len(accuarcy_list)
    if(acc>prev_acc):
        prev_acc = acc
        save(model, save_dir, save_prefix+"_"+str(round(acc,3)), i)

Epoch 0:  69%|██████▉   | 86/125 [01:22<00:37,  1.04batch/s, accuracy=97.65625, loss=0.216]


KeyboardInterrupt: ignored

# Load & Test

In [20]:
model.load_state_dict(torch.load("./data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"))

model.eval()

textCNN(
  (embed): Embedding(1481, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 256, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 256, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 256, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=768, out_features=4, bias=True)
)

In [21]:
%%time
q = "제주도 오늘 날씨 알려줘"

x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

intent = dataset.intents[torch.argmax(f).tolist()]

print("발화 : " + q)
print("의도 : " + intent)

발화 : 제주도 오늘 날씨 알려줘
의도 : weather
CPU times: user 7.54 ms, sys: 1.02 ms, total: 8.56 ms
Wall time: 17.8 ms
